<a href="https://colab.research.google.com/github/DavidMercadoFaustino/Modelagem-de-Previsao-de-Inadimplencia/blob/main/%20Engenharia-de-Features/%20%20Engenharia_de_Features_Home_Credit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install kaggle

In [2]:
! mkdir ~/.kaggle
! cp /content/kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [3]:
! kaggle competitions download -c home-credit-default-risk -p '/content/home-credit-default-risk'

home-credit-default-risk.zip: Skipping, found more recently modified local copy (use --force to force download)


In [4]:
import pandas as pd
import os
import zipfile


In [5]:
# Mostra para Python onde estão os arquivos do dataset
caminho_projeto = '/content/home-credit-default-risk'
if not os.path.exists(caminho_projeto):
    os.mkdir(caminho_projeto)
    print(f"Pasta criada em: {caminho_projeto}")
else:
    print(f"A pasta já existe em: {caminho_projeto}")

# Criar pasta para imagens se não existir
imagens_dir = 'imagens' # Nome da pasta para imagens
if not os.path.exists(imagens_dir):
    os.makedirs(imagens_dir)
    print(f"Pasta '{imagens_dir}' criada para salvar gráficos.")
else:
    print(f"Pasta '{imagens_dir}' já existe.")

caminho = os.getcwd()
# Todos arquivos NA PASTA INFORMADA
arquivos = os.listdir(caminho_projeto)
for arquivo in arquivos:
    if arquivo.endswith('.zip'):
        caminho_completo = os.path.join(caminho_projeto, arquivo)

A pasta já existe em: /content/home-credit-default-risk
Pasta 'imagens' já existe.


In [6]:
with zipfile.ZipFile(caminho_completo, 'r') as zip_ref:
    zip_ref.extractall(caminho_projeto)
print("Arquivos extraídos com sucesso.")

Arquivos extraídos com sucesso.


In [7]:
df_bureau = pd.read_csv(os.path.join(caminho_projeto, 'bureau.csv'))
print(df_bureau.head())
print(df_bureau.shape)


   SK_ID_CURR  SK_ID_BUREAU CREDIT_ACTIVE CREDIT_CURRENCY  DAYS_CREDIT  \
0      215354       5714462        Closed      currency 1         -497   
1      215354       5714463        Active      currency 1         -208   
2      215354       5714464        Active      currency 1         -203   
3      215354       5714465        Active      currency 1         -203   
4      215354       5714466        Active      currency 1         -629   

   CREDIT_DAY_OVERDUE  DAYS_CREDIT_ENDDATE  DAYS_ENDDATE_FACT  \
0                   0               -153.0             -153.0   
1                   0               1075.0                NaN   
2                   0                528.0                NaN   
3                   0                  NaN                NaN   
4                   0               1197.0                NaN   

   AMT_CREDIT_MAX_OVERDUE  CNT_CREDIT_PROLONG  AMT_CREDIT_SUM  \
0                     NaN                   0         91323.0   
1                     NaN         

In [8]:
bureau_agg = df_bureau.groupby('SK_ID_CURR').agg({
    'DAYS_CREDIT': ['mean', 'max', 'min'],
    'AMT_CREDIT_SUM': ['sum', 'mean'],
    'CREDIT_ACTIVE': ['count'] # Já fizemos, mas podemos adicionar outras features aqui
})

In [9]:
bureau_agg.head(10)

DAYS_CREDIT             AMT_CREDIT_SUM                 \
                   mean   max   min            sum           mean   
SK_ID_CURR                                                          
100001      -735.000000   -49 -1572    1453365.000  207623.571429   
100002      -874.000000  -103 -1437     865055.565  108131.945625   
100003     -1400.750000  -606 -2586    1017400.500  254350.125000   
100004      -867.000000  -408 -1326     189037.800   94518.900000   
100005      -190.666667   -62  -373     657126.000  219042.000000   
100007     -1149.000000 -1149 -1149     146250.000  146250.000000   
100008      -757.333333   -78 -1097     468445.500  156148.500000   
100009     -1271.500000  -239 -2882    4800811.500  266711.750000   
100010     -1939.500000 -1138 -2741     990000.000  495000.000000   
100011     -1773.000000 -1309 -2508     435228.300  108807.075000   

           CREDIT_ACTIVE  
                   count  
SK_ID_CURR                
100001                 7  
100002                 8  
100003                 4  
100004                 2  
100005                 3  
100007                 1  
100008                 3  
100009                18  
100010                 2  
100011                 4

In [10]:
# Renomear as colunas de forma mais limpa
bureau_agg.columns = [f'bureau_{col[0].upper()}_{col[1].upper()}' for col in bureau_agg.columns.values]
bureau_agg = bureau_agg.reset_index()
bureau_agg.head()


,SK_ID_CURR,bureau_DAYS_CREDIT_MEAN,bureau_DAYS_CREDIT_MAX,bureau_DAYS_CREDIT_MIN,bureau_AMT_CREDIT_SUM_SUM,bureau_AMT_CREDIT_SUM_MEAN,bureau_CREDIT_ACTIVE_COUNT
0,100001,-735.000000,-49,-1572,1453365.000,207623.571429,7
1,100002,-874.000000,-103,-1437,865055.565,108131.945625,8
2,100003,-1400.750000,-606,-2586,1017400.500,254350.125000,4
3,100004,-867.000000,-408,-1326,189037.800,94518.900000,2
4,100005,-190.666667,-62,-373,657126.000,219042.000000,3


In [11]:
bureau_counts = df_bureau.groupby('SK_ID_CURR')['SK_ID_BUREAU'].count().reset_index()
bureau_counts.columns = ['SK_ID_CURR', 'bureau_CREDIT_COUNT']
bureau_counts.head()

,SK_ID_CURR,bureau_CREDIT_COUNT
0,100001,7
1,100002,8
2,100003,4
3,100004,2
4,100005,3


In [12]:
bureau_merge = pd.merge(bureau_agg, bureau_counts, on='SK_ID_CURR', how='left')
bureau_merge.head(10)

,SK_ID_CURR,bureau_DAYS_CREDIT_MEAN,bureau_DAYS_CREDIT_MAX,bureau_DAYS_CREDIT_MIN,bureau_AMT_CREDIT_SUM_SUM,bureau_AMT_CREDIT_SUM_MEAN,bureau_CREDIT_ACTIVE_COUNT,bureau_CREDIT_COUNT
0,100001,-735.000000,-49,-1572,1453365.000,207623.571429,7,7
1,100002,-874.000000,-103,-1437,865055.565,108131.945625,8,8
2,100003,-1400.750000,-606,-2586,1017400.500,254350.125000,4,4
3,100004,-867.000000,-408,-1326,189037.800,94518.900000,2,2
4,100005,-190.666667,-62,-373,657126.000,219042.000000,3,3
5,100007,-1149.000000,-1149,-1149,146250.000,146250.000000,1,1
6,100008,-757.333333,-78,-1097,468445.500,156148.500000,3,3
7,100009,-1271.500000,-239,-2882,4800811.500,266711.750000,18,18
8,100010,-1939.500000,-1138,-2741,990000.000,495000.000000,2,2
9,100011,-1773.000000,-1309,-2508,435228.300,108807.075000,4,4


In [13]:
import data_preprocessing as dp

In [14]:
df= pd.read_csv("/content/application_train_processed.csv")

In [15]:
df_train_with_bureau = pd.merge(df, bureau_agg, on='SK_ID_CURR', how='left')
print("Shape do DataFrame após o merge com df_bureau:", df_train_with_bureau.shape)


Shape do DataFrame após o merge com df_bureau: (307511, 261)


In [16]:
# Mostra as novas colunas que foram adicionadas
print("\nNovas colunas adicionadas:")
print([col for col in df_train_with_bureau.columns if 'bureau' in col])

# Exibir as primeiras linhas para ver o resultado do merge
print("\nDataFrame após o merge com as features do bureau:")
print(df_train_with_bureau.head())


Novas colunas adicionadas:
['bureau_DAYS_CREDIT_MEAN', 'bureau_DAYS_CREDIT_MAX', 'bureau_DAYS_CREDIT_MIN', 'bureau_AMT_CREDIT_SUM_SUM', 'bureau_AMT_CREDIT_SUM_MEAN', 'bureau_CREDIT_ACTIVE_COUNT']

DataFrame após o merge com as features do bureau:
   SK_ID_CURR  TARGET  CNT_CHILDREN  AMT_INCOME_TOTAL  AMT_CREDIT  \
0      100002       1             0          202500.0    406597.5   
1      100003       0             0          270000.0   1293502.5   
2      100004       0             0           67500.0    135000.0   
3      100006       0             0          135000.0    312682.5   
4      100007       0             0          121500.0    513000.0   

   AMT_ANNUITY  AMT_GOODS_PRICE  REGION_POPULATION_RELATIVE  DAYS_BIRTH  \
0      24700.5         351000.0                    0.018801       -9461   
1      35698.5        1129500.0                    0.003541      -16765   
2       6750.0         135000.0                    0.010032      -19046   
3      29686.5         297000.0      

In [17]:
print(df_train_with_bureau[['bureau_DAYS_CREDIT_MEAN', 'bureau_AMT_CREDIT_SUM_SUM']].isnull().sum())


bureau_DAYS_CREDIT_MEAN      44020
bureau_AMT_CREDIT_SUM_SUM    44020
dtype: int64


In [18]:
# Preencher os NaNs nas colunas agregadas com 0
for col in bureau_agg.columns:
    if col != 'SK_ID_CURR': # Não queremos preencher a coluna de ID
        df_train_with_bureau[col] = df_train_with_bureau[col].fillna(0)

print(df_train_with_bureau.head())

   SK_ID_CURR  TARGET  CNT_CHILDREN  AMT_INCOME_TOTAL  AMT_CREDIT  \
0      100002       1             0          202500.0    406597.5   
1      100003       0             0          270000.0   1293502.5   
2      100004       0             0           67500.0    135000.0   
3      100006       0             0          135000.0    312682.5   
4      100007       0             0          121500.0    513000.0   

   AMT_ANNUITY  AMT_GOODS_PRICE  REGION_POPULATION_RELATIVE  DAYS_BIRTH  \
0      24700.5         351000.0                    0.018801       -9461   
1      35698.5        1129500.0                    0.003541      -16765   
2       6750.0         135000.0                    0.010032      -19046   
3      29686.5         297000.0                    0.008019      -19005   
4      21865.5         513000.0                    0.028663      -19932   

   DAYS_EMPLOYED  ...  WALLSMATERIAL_MODE_Wooden  EMERGENCYSTATE_MODE_Missing  \
0         -637.0  ...                      False     

In [19]:
    df_bureau = pd.read_csv(os.path.join(caminho_projeto, 'bureau.csv'))
    df_bureau.head()

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,-20,NaN
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit card,-16,NaN
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,-21,NaN


In [20]:
bureau_agg = df_bureau.groupby('SK_ID_CURR').agg({
    'DAYS_CREDIT': ['mean', 'max', 'min'],
    'AMT_CREDIT_SUM': ['sum', 'mean']
})
bureau_agg.columns = [f'bureau_{col[0].upper()}_{col[1].upper()}' for col in bureau_agg.columns.values]
bureau_agg = bureau_agg.reset_index()
bureau_agg.head()

,SK_ID_CURR,bureau_DAYS_CREDIT_MEAN,bureau_DAYS_CREDIT_MAX,bureau_DAYS_CREDIT_MIN,bureau_AMT_CREDIT_SUM_SUM,bureau_AMT_CREDIT_SUM_MEAN
0,100001,-735.000000,-49,-1572,1453365.000,207623.571429
1,100002,-874.000000,-103,-1437,865055.565,108131.945625
2,100003,-1400.750000,-606,-2586,1017400.500,254350.125000
3,100004,-867.000000,-408,-1326,189037.800,94518.900000
4,100005,-190.666667,-62,-373,657126.000,219042.000000


In [21]:
df_bureau_balance = pd.read_csv(os.path.join(caminho_projeto, 'bureau_balance.csv'))

df_bureau_balance = pd.get_dummies(df_bureau_balance, columns=['STATUS'], dummy_na=False)
df_bureau_balance.head()


,SK_ID_BUREAU,MONTHS_BALANCE,STATUS_0,STATUS_1,STATUS_2,STATUS_3,STATUS_4,STATUS_5,STATUS_C,STATUS_X
0,5715448,0,False,False,False,False,False,False,True,False
1,5715448,-1,False,False,False,False,False,False,True,False
2,5715448,-2,False,False,False,False,False,False,True,False
3,5715448,-3,False,False,False,False,False,False,True,False
4,5715448,-4,False,False,False,False,False,False,True,False


In [22]:
df_bureau_balance.columns

Index(['SK_ID_BUREAU', 'MONTHS_BALANCE', 'STATUS_0', 'STATUS_1', 'STATUS_2',
       'STATUS_3', 'STATUS_4', 'STATUS_5', 'STATUS_C', 'STATUS_X'],
      dtype='object')

In [23]:
bureau_balance_agg = df_bureau_balance.groupby('SK_ID_BUREAU').agg({
    'MONTHS_BALANCE': ['count', 'mean', 'min', 'max'],
    'STATUS_0': ['mean'], # Proporção de meses com status 0
    'STATUS_1': ['mean'], # Proporção de meses com atraso 1-30
    'STATUS_2': ['mean'], # Proporção de meses com atraso 31-60
    'STATUS_3': ['mean'], # Proporção de meses com atraso 61-90
    'STATUS_4': ['mean'], # Proporção de meses com atraso 91-120
    'STATUS_5': ['mean'], # Proporção de meses com atraso +120
    'STATUS_C': ['mean'], # Proporção de meses com conta fechada
    'STATUS_X': ['mean']  # Proporção de meses com status desconhecido
})

# Renomear as colunas de forma limpa novamente
bureau_balance_agg.columns = [f'bureau_balance_{col[0].upper()}_{col[1].upper()}' for col in bureau_balance_agg.columns.values]
bureau_balance_agg = bureau_balance_agg.reset_index()
bureau_balance_agg.head()

,SK_ID_BUREAU,bureau_balance_MONTHS_BALANCE_COUNT,bureau_balance_MONTHS_BALANCE_MEAN,bureau_balance_MONTHS_BALANCE_MIN,bureau_balance_MONTHS_BALANCE_MAX,bureau_balance_STATUS_0_MEAN,bureau_balance_STATUS_1_MEAN,bureau_balance_STATUS_2_MEAN,bureau_balance_STATUS_3_MEAN,bureau_balance_STATUS_4_MEAN,bureau_balance_STATUS_5_MEAN,bureau_balance_STATUS_C_MEAN,bureau_balance_STATUS_X_MEAN
0,5001709,97,-48.0,-96,0,0.000000,0.0,0.0,0.0,0.0,0.0,0.886598,0.113402
1,5001710,83,-41.0,-82,0,0.060241,0.0,0.0,0.0,0.0,0.0,0.578313,0.361446
2,5001711,4,-1.5,-3,0,0.750000,0.0,0.0,0.0,0.0,0.0,0.000000,0.250000
3,5001712,19,-9.0,-18,0,0.526316,0.0,0.0,0.0,0.0,0.0,0.473684,0.000000
4,5001713,22,-10.5,-21,0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,1.000000


In [24]:
df_bureau_merged = pd.merge(df_bureau, bureau_balance_agg, on='SK_ID_BUREAU', how='left')
# Preencher os NaNs nas colunas agregadas com 0
for col in df_bureau_merged.columns:
    if col != 'SK_ID_CURR': # Não queremos preencher a coluna de ID
        df_bureau_merged[col] = df_bureau_merged[col].fillna(0)
df_bureau_merged.head()

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,...,bureau_balance_MONTHS_BALANCE_MIN,bureau_balance_MONTHS_BALANCE_MAX,bureau_balance_STATUS_0_MEAN,bureau_balance_STATUS_1_MEAN,bureau_balance_STATUS_2_MEAN,bureau_balance_STATUS_3_MEAN,bureau_balance_STATUS_4_MEAN,bureau_balance_STATUS_5_MEAN,bureau_balance_STATUS_C_MEAN,bureau_balance_STATUS_X_MEAN
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,215354,5714463,Active,currency 1,-208,0,1075.0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,215354,5714464,Active,currency 1,-203,0,528.0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,215354,5714465,Active,currency 1,-203,0,0.0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,215354,5714466,Active,currency 1,-629,0,1197.0,0.0,77674.5,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
df_bureau_merged =pd.get_dummies(df_bureau_merged, columns=['CREDIT_ACTIVE','CREDIT_CURRENCY'], dummy_na=False)
bureau_final_agg = df_bureau_merged.groupby('SK_ID_CURR').agg('mean', numeric_only=True)
bureau_final_agg = bureau_final_agg.drop(columns=['SK_ID_BUREAU']).reset_index()
bureau_final_agg.head()

,SK_ID_CURR,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,...,bureau_balance_STATUS_C_MEAN,bureau_balance_STATUS_X_MEAN,CREDIT_ACTIVE_Active,CREDIT_ACTIVE_Bad debt,CREDIT_ACTIVE_Closed,CREDIT_ACTIVE_Sold,CREDIT_CURRENCY_currency 1,CREDIT_CURRENCY_currency 2,CREDIT_CURRENCY_currency 3,CREDIT_CURRENCY_currency 4
0,100001,-735.000000,0.0,82.428571,-471.714286,0.000000,0.0,207623.571429,85240.928571,0.000000,...,0.441240,0.214590,0.428571,0.0,0.571429,0.0,1.0,0.0,0.0,0.0
1,100002,-874.000000,0.0,-261.750000,-523.125000,1050.643125,0.0,108131.945625,30722.625000,3998.570625,...,0.175426,0.161932,0.250000,0.0,0.750000,0.0,1.0,0.0,0.0,0.0
2,100003,-1400.750000,0.0,-544.500000,-823.000000,0.000000,0.0,254350.125000,0.000000,202500.000000,...,0.000000,0.000000,0.250000,0.0,0.750000,0.0,1.0,0.0,0.0,0.0
3,100004,-867.000000,0.0,-488.500000,-532.500000,0.000000,0.0,94518.900000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,1.000000,0.0,1.0,0.0,0.0,0.0
4,100005,-190.666667,0.0,439.333333,-41.000000,0.000000,0.0,219042.000000,189469.500000,0.000000,...,0.128205,0.136752,0.666667,0.0,0.333333,0.0,1.0,0.0,0.0,0.0


In [26]:
final_application = pd.merge(df_train_with_bureau, bureau_final_agg, on='SK_ID_CURR', how='left')
# Preencher os NaNs nas colunas agregadas com 0
for col in final_application.columns:
    if col != 'SK_ID_CURR': # Não queremos preencher a coluna de ID
        final_application[col] = final_application[col].fillna(0)
final_application.head()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY_x,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,bureau_balance_STATUS_C_MEAN,bureau_balance_STATUS_X_MEAN,CREDIT_ACTIVE_Active,CREDIT_ACTIVE_Bad debt,CREDIT_ACTIVE_Closed,CREDIT_ACTIVE_Sold,CREDIT_CURRENCY_currency 1,CREDIT_CURRENCY_currency 2,CREDIT_CURRENCY_currency 3,CREDIT_CURRENCY_currency 4
0,100002,1,0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461,-637.0,...,0.175426,0.161932,0.25,0.0,0.75,0.0,1.0,0.0,0.0,0.0
1,100003,0,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765,-1188.0,...,0.000000,0.000000,0.25,0.0,0.75,0.0,1.0,0.0,0.0,0.0
2,100004,0,0,67500.0,135000.0,6750.0,135000.0,0.010032,-19046,-225.0,...,0.000000,0.000000,0.00,0.0,1.00,0.0,1.0,0.0,0.0,0.0
3,100006,0,0,135000.0,312682.5,29686.5,297000.0,0.008019,-19005,-3039.0,...,0.000000,0.000000,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0
4,100007,0,0,121500.0,513000.0,21865.5,513000.0,0.028663,-19932,-3038.0,...,0.000000,0.000000,0.00,0.0,1.00,0.0,1.0,0.0,0.0,0.0


In [27]:
print(final_application.columns)

Index(['SK_ID_CURR', 'TARGET', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL',
       'AMT_CREDIT', 'AMT_ANNUITY_x', 'AMT_GOODS_PRICE',
       'REGION_POPULATION_RELATIVE', 'DAYS_BIRTH', 'DAYS_EMPLOYED',
       ...
       'bureau_balance_STATUS_C_MEAN', 'bureau_balance_STATUS_X_MEAN',
       'CREDIT_ACTIVE_Active', 'CREDIT_ACTIVE_Bad debt',
       'CREDIT_ACTIVE_Closed', 'CREDIT_ACTIVE_Sold',
       'CREDIT_CURRENCY_currency 1', 'CREDIT_CURRENCY_currency 2',
       'CREDIT_CURRENCY_currency 3', 'CREDIT_CURRENCY_currency 4'],
      dtype='object', length=293)


In [28]:
 df_previous = pd.read_csv(os.path.join(caminho_projeto, 'previous_application.csv'))
 df_previous.head()

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,...,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
0,2030495,271877,Consumer loans,1730.430,17145.0,17145.0,0.0,17145.0,SATURDAY,15,...,Connectivity,12.0,middle,POS mobile with interest,365243.0,-42.0,300.0,-42.0,-37.0,0.0
1,2802425,108129,Cash loans,25188.615,607500.0,679671.0,NaN,607500.0,THURSDAY,11,...,XNA,36.0,low_action,Cash X-Sell: low,365243.0,-134.0,916.0,365243.0,365243.0,1.0
2,2523466,122040,Cash loans,15060.735,112500.0,136444.5,NaN,112500.0,TUESDAY,11,...,XNA,12.0,high,Cash X-Sell: high,365243.0,-271.0,59.0,365243.0,365243.0,1.0
3,2819243,176158,Cash loans,47041.335,450000.0,470790.0,NaN,450000.0,MONDAY,7,...,XNA,12.0,middle,Cash X-Sell: middle,365243.0,-482.0,-152.0,-182.0,-177.0,1.0
4,1784265,202054,Cash loans,31924.395,337500.0,404055.0,NaN,337500.0,THURSDAY,9,...,XNA,24.0,high,Cash Street: high,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
print(df_previous['NAME_PORTFOLIO'].value_counts())

NAME_PORTFOLIO
POS      691011
Cash     461563
XNA      372230
Cards    144985
Cars        425
Name: count, dtype: int64


In [30]:
df_previous = pd.get_dummies(df_previous, columns=['NAME_CONTRACT_STATUS','NAME_YIELD_GROUP','PRODUCT_COMBINATION','CHANNEL_TYPE','NAME_CLIENT_TYPE','NAME_PAYMENT_TYPE','NAME_PORTFOLIO','NAME_PRODUCT_TYPE'], dummy_na=False)
df_previous

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,...,NAME_PAYMENT_TYPE_Non-cash from your account,NAME_PAYMENT_TYPE_XNA,NAME_PORTFOLIO_Cards,NAME_PORTFOLIO_Cars,NAME_PORTFOLIO_Cash,NAME_PORTFOLIO_POS,NAME_PORTFOLIO_XNA,NAME_PRODUCT_TYPE_XNA,NAME_PRODUCT_TYPE_walk-in,NAME_PRODUCT_TYPE_x-sell
0,2030495,271877,Consumer loans,1730.430,17145.0,17145.0,0.0,17145.0,SATURDAY,15,...,False,False,False,False,False,True,False,True,False,False
1,2802425,108129,Cash loans,25188.615,607500.0,679671.0,NaN,607500.0,THURSDAY,11,...,False,True,False,False,True,False,False,False,False,True
2,2523466,122040,Cash loans,15060.735,112500.0,136444.5,NaN,112500.0,TUESDAY,11,...,False,False,False,False,True,False,False,False,False,True
3,2819243,176158,Cash loans,47041.335,450000.0,470790.0,NaN,450000.0,MONDAY,7,...,False,False,False,False,True,False,False,False,False,True
4,1784265,202054,Cash loans,31924.395,337500.0,404055.0,NaN,337500.0,THURSDAY,9,...,False,False,False,False,True,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1670209,2300464,352015,Consumer loans,14704.290,267295.5,311400.0,0.0,267295.5,WEDNESDAY,12,...,False,False,False,False,False,True,False,True,False,False
1670210,2357031,334635,Consumer loans,6622.020,87750.0,64291.5,29250.0,87750.0,TUESDAY,15,...,False,False,False,False,False,True,False,True,False,False
1670211,2659632,249544,Consumer loans,11520.855,105237.0,102523.5,10525.5,105237.0,MONDAY,12,...,False,False,False,False,False,True,False,True,False,False
1670212,2785582,400317,Cash loans,18821.520,180000.0,191880.0,NaN,180000.0,WEDNESDAY,9,...,False,False,False,False,True,False,False,False,False,True


In [31]:
df_previous.columns

Index(['SK_ID_PREV', 'SK_ID_CURR', 'NAME_CONTRACT_TYPE', 'AMT_ANNUITY',
       'AMT_APPLICATION', 'AMT_CREDIT', 'AMT_DOWN_PAYMENT', 'AMT_GOODS_PRICE',
       'WEEKDAY_APPR_PROCESS_START', 'HOUR_APPR_PROCESS_START',
       'FLAG_LAST_APPL_PER_CONTRACT', 'NFLAG_LAST_APPL_IN_DAY',
       'RATE_DOWN_PAYMENT', 'RATE_INTEREST_PRIMARY',
       'RATE_INTEREST_PRIVILEGED', 'NAME_CASH_LOAN_PURPOSE', 'DAYS_DECISION',
       'CODE_REJECT_REASON', 'NAME_TYPE_SUITE', 'NAME_GOODS_CATEGORY',
       'SELLERPLACE_AREA', 'NAME_SELLER_INDUSTRY', 'CNT_PAYMENT',
       'DAYS_FIRST_DRAWING', 'DAYS_FIRST_DUE', 'DAYS_LAST_DUE_1ST_VERSION',
       'DAYS_LAST_DUE', 'DAYS_TERMINATION', 'NFLAG_INSURED_ON_APPROVAL',
       'NAME_CONTRACT_STATUS_Approved', 'NAME_CONTRACT_STATUS_Canceled',
       'NAME_CONTRACT_STATUS_Refused', 'NAME_CONTRACT_STATUS_Unused offer',
       'NAME_YIELD_GROUP_XNA', 'NAME_YIELD_GROUP_high',
       'NAME_YIELD_GROUP_low_action', 'NAME_YIELD_GROUP_low_normal',
       'NAME_YIELD_GROUP_middl

In [32]:
prev_agg = df_previous.groupby('SK_ID_CURR').agg({
    'SK_ID_PREV': ['count'],
    'AMT_CREDIT': ['mean', 'max', 'sum'],
    'DAYS_DECISION': ['mean', 'min', 'max'],
    'NAME_CONTRACT_STATUS_Approved': ['mean'], # Proporção de aprovações
    'NAME_CONTRACT_STATUS_Canceled': ['mean'],
    'NAME_CONTRACT_STATUS_Refused': ['mean'],
    'NAME_CONTRACT_STATUS_Unused offer': ['mean'],
    'NAME_PAYMENT_TYPE_Cash through the bank': ['mean'],
    'NAME_PAYMENT_TYPE_Cashless from the account of the employer': ['mean'],
    'NAME_PAYMENT_TYPE_Non-cash from your account': ['mean'],
    'NAME_PAYMENT_TYPE_XNA': ['mean'],
    'NAME_PORTFOLIO_Cards': ['mean'],
    'NAME_PORTFOLIO_Cash': ['mean'],
    'NAME_PORTFOLIO_POS': ['mean'],
    'NAME_PORTFOLIO_XNA': ['mean'],
    'NAME_PRODUCT_TYPE_walk-in': ['mean'],
    'NAME_PRODUCT_TYPE_x-sell': ['mean'],
    'CHANNEL_TYPE_AP+ (Cash loan)': ['mean'],
    'CHANNEL_TYPE_Car dealer': ['mean'],
    'CHANNEL_TYPE_Channel of corporate sales': ['mean'],
    'CHANNEL_TYPE_Contact center': ['mean'],
    'CHANNEL_TYPE_Country-wide': ['mean'],
    'CHANNEL_TYPE_Credit and cash offices': ['mean'],
    'CHANNEL_TYPE_Regional / Local': ['mean'],
    'CHANNEL_TYPE_Stone': ['mean'],
    'NAME_CLIENT_TYPE_New': ['mean'],
    'NAME_CLIENT_TYPE_Refreshed': ['mean'],
    'NAME_CLIENT_TYPE_Repeater': ['mean'],
    'NAME_CLIENT_TYPE_XNA': ['mean'],
    'NAME_YIELD_GROUP_high': ['mean'],
    'NAME_YIELD_GROUP_low_action': ['mean'],
    'NAME_YIELD_GROUP_low_normal': ['mean'],
    'NAME_YIELD_GROUP_middle': ['mean'],
    'PRODUCT_COMBINATION_Card Street': ['mean'],
    'PRODUCT_COMBINATION_Card X-Sell': ['mean'],
    'PRODUCT_COMBINATION_Cash': ['mean'],
    'PRODUCT_COMBINATION_Cash Street: high': ['mean'],
    'PRODUCT_COMBINATION_Cash Street: low': ['mean'],
    'PRODUCT_COMBINATION_Cash Street: middle': ['mean'],
    'PRODUCT_COMBINATION_Cash X-Sell: high': ['mean'],
    'PRODUCT_COMBINATION_Cash X-Sell: low': ['mean'],
    'PRODUCT_COMBINATION_Cash X-Sell: middle': ['mean'],
    'PRODUCT_COMBINATION_POS household with interest': ['mean'],
    'PRODUCT_COMBINATION_POS household without interest': ['mean'],
    'PRODUCT_COMBINATION_POS industry with interest': ['mean'],
    'PRODUCT_COMBINATION_POS industry without interest': ['mean'],
    'PRODUCT_COMBINATION_POS mobile with interest': ['mean'],
    'PRODUCT_COMBINATION_POS mobile without interest': ['mean'],
    'PRODUCT_COMBINATION_POS other with interest': ['mean'],
    'PRODUCT_COMBINATION_POS others without interest': ['mean']
})
prev_agg.columns = [f'prev_{col[0].upper()}_{col[1].upper()}' for col in prev_agg.columns.values]
prev_agg = prev_agg.reset_index()
prev_agg.head()

,SK_ID_CURR,prev_SK_ID_PREV_COUNT,prev_AMT_CREDIT_MEAN,prev_AMT_CREDIT_MAX,prev_AMT_CREDIT_SUM,prev_DAYS_DECISION_MEAN,prev_DAYS_DECISION_MIN,prev_DAYS_DECISION_MAX,prev_NAME_CONTRACT_STATUS_APPROVED_MEAN,prev_NAME_CONTRACT_STATUS_CANCELED_MEAN,...,prev_PRODUCT_COMBINATION_CASH X-SELL: LOW_MEAN,prev_PRODUCT_COMBINATION_CASH X-SELL: MIDDLE_MEAN,prev_PRODUCT_COMBINATION_POS HOUSEHOLD WITH INTEREST_MEAN,prev_PRODUCT_COMBINATION_POS HOUSEHOLD WITHOUT INTEREST_MEAN,prev_PRODUCT_COMBINATION_POS INDUSTRY WITH INTEREST_MEAN,prev_PRODUCT_COMBINATION_POS INDUSTRY WITHOUT INTEREST_MEAN,prev_PRODUCT_COMBINATION_POS MOBILE WITH INTEREST_MEAN,prev_PRODUCT_COMBINATION_POS MOBILE WITHOUT INTEREST_MEAN,prev_PRODUCT_COMBINATION_POS OTHER WITH INTEREST_MEAN,prev_PRODUCT_COMBINATION_POS OTHERS WITHOUT INTEREST_MEAN
0,100001,1,23787.00,23787.0,23787.0,-1740.0,-1740,-1740,1.0,0.0,...,0.000000,0.0,0.000000,0.0,0.000000,0.0,1.0,0.0,0.0,0.0
1,100002,1,179055.00,179055.0,179055.0,-606.0,-606,-606,1.0,0.0,...,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,1.0,0.0
2,100003,3,484191.00,1035882.0,1452573.0,-1305.0,-2341,-746,1.0,0.0,...,0.333333,0.0,0.333333,0.0,0.333333,0.0,0.0,0.0,0.0,0.0
3,100004,1,20106.00,20106.0,20106.0,-815.0,-815,-815,1.0,0.0,...,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,1.0,0.0,0.0
4,100005,2,20076.75,40153.5,40153.5,-536.0,-757,-315,0.5,0.5,...,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.5,0.0,0.0,0.0


In [33]:
df_train_prev = pd.merge(final_application, prev_agg, on='SK_ID_CURR', how='left')
for col in prev_agg.columns:
    if col != 'SK_ID_CURR':
        df_train_prev[col].fillna(0, inplace=True)
df_train_prev.head()

/tmp/ipython-input-1324668332.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_train_prev[col].fillna(0, inplace=True)


,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY_x,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,prev_PRODUCT_COMBINATION_CASH X-SELL: LOW_MEAN,prev_PRODUCT_COMBINATION_CASH X-SELL: MIDDLE_MEAN,prev_PRODUCT_COMBINATION_POS HOUSEHOLD WITH INTEREST_MEAN,prev_PRODUCT_COMBINATION_POS HOUSEHOLD WITHOUT INTEREST_MEAN,prev_PRODUCT_COMBINATION_POS INDUSTRY WITH INTEREST_MEAN,prev_PRODUCT_COMBINATION_POS INDUSTRY WITHOUT INTEREST_MEAN,prev_PRODUCT_COMBINATION_POS MOBILE WITH INTEREST_MEAN,prev_PRODUCT_COMBINATION_POS MOBILE WITHOUT INTEREST_MEAN,prev_PRODUCT_COMBINATION_POS OTHER WITH INTEREST_MEAN,prev_PRODUCT_COMBINATION_POS OTHERS WITHOUT INTEREST_MEAN
0,100002,1,0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461,-637.0,...,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,1.0,0.0
1,100003,0,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765,-1188.0,...,0.333333,0.0,0.333333,0.0,0.333333,0.0,0.000000,0.0,0.0,0.0
2,100004,0,0,67500.0,135000.0,6750.0,135000.0,0.010032,-19046,-225.0,...,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,1.0,0.0,0.0
3,100006,0,0,135000.0,312682.5,29686.5,297000.0,0.008019,-19005,-3039.0,...,0.222222,0.0,0.111111,0.0,0.111111,0.0,0.000000,0.0,0.0,0.0
4,100007,0,0,121500.0,513000.0,21865.5,513000.0,0.028663,-19932,-3038.0,...,0.000000,0.5,0.166667,0.0,0.000000,0.0,0.166667,0.0,0.0,0.0


In [34]:
df_pos_cash = pd.read_csv(os.path.join(caminho_projeto, 'POS_CASH_balance.csv'))
df_pos_cash.head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,1803195,182943,-31,48.0,45.0,Active,0,0
1,1715348,367990,-33,36.0,35.0,Active,0,0
2,1784872,397406,-32,12.0,9.0,Active,0,0
3,1903291,269225,-35,48.0,42.0,Active,0,0
4,2341044,334279,-35,36.0,35.0,Active,0,0


In [35]:
df_pos_cash = pd.get_dummies(df_pos_cash, columns=['NAME_CONTRACT_STATUS'], dummy_na=False)
pos_cash_agg = df_pos_cash.groupby('SK_ID_CURR').agg({
    'SK_ID_PREV': ['count'], # Contagem de extratos mensais
    'MONTHS_BALANCE': ['mean', 'min', 'max'],
    'SK_DPD': ['mean', 'max'],
    'SK_DPD_DEF': ['mean', 'max'],
    'NAME_CONTRACT_STATUS_Active': ['mean'],
    'NAME_CONTRACT_STATUS_Completed': ['mean']
})
pos_cash_agg.columns = [f'pos_cash_{col[0].upper()}_{col[1].upper()}' for col in pos_cash_agg.columns.values]
pos_cash_agg = pos_cash_agg.reset_index()
pos_cash_agg.head()


,SK_ID_CURR,pos_cash_SK_ID_PREV_COUNT,pos_cash_MONTHS_BALANCE_MEAN,pos_cash_MONTHS_BALANCE_MIN,pos_cash_MONTHS_BALANCE_MAX,pos_cash_SK_DPD_MEAN,pos_cash_SK_DPD_MAX,pos_cash_SK_DPD_DEF_MEAN,pos_cash_SK_DPD_DEF_MAX,pos_cash_NAME_CONTRACT_STATUS_ACTIVE_MEAN,pos_cash_NAME_CONTRACT_STATUS_COMPLETED_MEAN
0,100001,9,-72.555556,-96,-53,0.777778,7,0.777778,7,0.777778,0.222222
1,100002,19,-10.000000,-19,-1,0.000000,0,0.000000,0,1.000000,0.000000
2,100003,28,-43.785714,-77,-18,0.000000,0,0.000000,0,0.928571,0.071429
3,100004,4,-25.500000,-27,-24,0.000000,0,0.000000,0,0.750000,0.250000
4,100005,11,-20.000000,-25,-15,0.000000,0,0.000000,0,0.818182,0.090909


In [36]:
df_train_cash = pd.merge(df_train_prev, pos_cash_agg, on='SK_ID_CURR', how='left')
for col in pos_cash_agg.columns:
    if col != 'SK_ID_CURR':
        df_train_cash[col].fillna(0, inplace=True)
df_train_cash.head()

/tmp/ipython-input-2800605440.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_train_cash[col].fillna(0, inplace=True)


,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY_x,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,pos_cash_SK_ID_PREV_COUNT,pos_cash_MONTHS_BALANCE_MEAN,pos_cash_MONTHS_BALANCE_MIN,pos_cash_MONTHS_BALANCE_MAX,pos_cash_SK_DPD_MEAN,pos_cash_SK_DPD_MAX,pos_cash_SK_DPD_DEF_MEAN,pos_cash_SK_DPD_DEF_MAX,pos_cash_NAME_CONTRACT_STATUS_ACTIVE_MEAN,pos_cash_NAME_CONTRACT_STATUS_COMPLETED_MEAN
0,100002,1,0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461,-637.0,...,19.0,-10.000000,-19.0,-1.0,0.0,0.0,0.0,0.0,1.000000,0.000000
1,100003,0,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765,-1188.0,...,28.0,-43.785714,-77.0,-18.0,0.0,0.0,0.0,0.0,0.928571,0.071429
2,100004,0,0,67500.0,135000.0,6750.0,135000.0,0.010032,-19046,-225.0,...,4.0,-25.500000,-27.0,-24.0,0.0,0.0,0.0,0.0,0.750000,0.250000
3,100006,0,0,135000.0,312682.5,29686.5,297000.0,0.008019,-19005,-3039.0,...,21.0,-9.619048,-20.0,-1.0,0.0,0.0,0.0,0.0,0.857143,0.095238
4,100007,0,0,121500.0,513000.0,21865.5,513000.0,0.028663,-19932,-3038.0,...,66.0,-33.636364,-77.0,-1.0,0.0,0.0,0.0,0.0,0.939394,0.045455


In [37]:
df_installments = pd.read_csv(os.path.join(caminho_projeto, 'installments_payments.csv'))
df_installments.head()

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
0,1054186,161674,1.0,6,-1180.0,-1187.0,6948.360,6948.360
1,1330831,151639,0.0,34,-2156.0,-2156.0,1716.525,1716.525
2,2085231,193053,2.0,1,-63.0,-63.0,25425.000,25425.000
3,2452527,199697,1.0,3,-2418.0,-2426.0,24350.130,24350.130
4,2714724,167756,1.0,2,-1383.0,-1366.0,2165.040,2160.585


In [38]:
df_installments['diff_installments'] = df_installments['DAYS_INSTALMENT'] - df_installments['DAYS_ENTRY_PAYMENT']

In [39]:
df_installments_agg = df_installments.groupby('SK_ID_CURR').agg({
    'SK_ID_PREV': ['count'], # Contagem de extratos mensais
    'AMT_INSTALMENT': ['mean', 'min', 'max'],
    'AMT_PAYMENT': ['mean', 'min', 'max'],
    'diff_installments': ['mean', 'min', 'max']
})
df_installments_agg.columns = [f'installments_{col[0].upper()}_{col[1].upper()}' for col in df_installments_agg.columns.values]
df_installments_agg = df_installments_agg.reset_index()
df_installments_agg.head()

,SK_ID_CURR,installments_SK_ID_PREV_COUNT,installments_AMT_INSTALMENT_MEAN,installments_AMT_INSTALMENT_MIN,installments_AMT_INSTALMENT_MAX,installments_AMT_PAYMENT_MEAN,installments_AMT_PAYMENT_MIN,installments_AMT_PAYMENT_MAX,installments_DIFF_INSTALLMENTS_MEAN,installments_DIFF_INSTALLMENTS_MIN,installments_DIFF_INSTALLMENTS_MAX
0,100001,7,5885.132143,3951.000,17397.900,5885.132143,3951.000,17397.900,7.285714,-11.0,36.0
1,100002,19,11559.247105,9251.775,53093.745,11559.247105,9251.775,53093.745,20.421053,12.0,31.0
2,100003,25,64754.586000,6662.970,560835.360,64754.586000,6662.970,560835.360,7.160000,1.0,14.0
3,100004,3,7096.155000,5357.250,10573.965,7096.155000,5357.250,10573.965,7.666667,3.0,11.0
4,100005,9,6240.205000,4813.200,17656.245,6240.205000,4813.200,17656.245,23.555556,-1.0,37.0


In [40]:
df_train_installments = pd.merge(df_train_cash, df_installments_agg, on='SK_ID_CURR', how='left')
for col in df_installments_agg.columns:
    if col != 'SK_ID_CURR':
        df_train_installments[col].fillna(0, inplace=True)
df_train_installments.head()

/tmp/ipython-input-3247063255.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_train_installments[col].fillna(0, inplace=True)


,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY_x,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,installments_SK_ID_PREV_COUNT,installments_AMT_INSTALMENT_MEAN,installments_AMT_INSTALMENT_MIN,installments_AMT_INSTALMENT_MAX,installments_AMT_PAYMENT_MEAN,installments_AMT_PAYMENT_MIN,installments_AMT_PAYMENT_MAX,installments_DIFF_INSTALLMENTS_MEAN,installments_DIFF_INSTALLMENTS_MIN,installments_DIFF_INSTALLMENTS_MAX
0,100002,1,0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461,-637.0,...,19.0,11559.247105,9251.775,53093.745,11559.247105,9251.775,53093.745,20.421053,12.0,31.0
1,100003,0,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765,-1188.0,...,25.0,64754.586000,6662.970,560835.360,64754.586000,6662.970,560835.360,7.160000,1.0,14.0
2,100004,0,0,67500.0,135000.0,6750.0,135000.0,0.010032,-19046,-225.0,...,3.0,7096.155000,5357.250,10573.965,7096.155000,5357.250,10573.965,7.666667,3.0,11.0
3,100006,0,0,135000.0,312682.5,29686.5,297000.0,0.008019,-19005,-3039.0,...,16.0,62947.088438,2482.920,691786.890,62947.088438,2482.920,691786.890,19.375000,1.0,77.0
4,100007,0,0,121500.0,513000.0,21865.5,513000.0,0.028663,-19932,-3038.0,...,66.0,12666.444545,1821.780,22678.785,12214.060227,0.180,22678.785,3.636364,-12.0,31.0


In [41]:
df_credit_card = pd.read_csv(os.path.join(caminho_projeto, 'credit_card_balance.csv'))
df_credit_card.head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,...,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,2562384,378907,-6,56.970,135000,0.0,877.5,0.0,877.5,1700.325,...,0.000,0.000,0.0,1,0.0,1.0,35.0,Active,0,0
1,2582071,363914,-1,63975.555,45000,2250.0,2250.0,0.0,0.0,2250.000,...,64875.555,64875.555,1.0,1,0.0,0.0,69.0,Active,0,0
2,1740877,371185,-7,31815.225,450000,0.0,0.0,0.0,0.0,2250.000,...,31460.085,31460.085,0.0,0,0.0,0.0,30.0,Active,0,0
3,1389973,337855,-4,236572.110,225000,2250.0,2250.0,0.0,0.0,11795.760,...,233048.970,233048.970,1.0,1,0.0,0.0,10.0,Active,0,0
4,1891521,126868,-1,453919.455,450000,0.0,11547.0,0.0,11547.0,22924.890,...,453919.455,453919.455,0.0,1,0.0,1.0,101.0,Active,0,0


In [42]:
df_credit_card = pd.get_dummies(df_credit_card, columns=['NAME_CONTRACT_STATUS'], dummy_na=False)
credit_card_agg = df_credit_card.groupby('SK_ID_CURR').agg({
    'SK_ID_PREV': ['count'], # Contagem de extratos mensais
    'MONTHS_BALANCE': ['mean', 'min', 'max'],
    'AMT_BALANCE': ['mean', 'max', 'min'],
    'AMT_CREDIT_LIMIT_ACTUAL': ['mean', 'max', 'min'],
    'AMT_DRAWINGS_ATM_CURRENT': ['mean', 'max', 'min'],
    'AMT_DRAWINGS_CURRENT': ['mean', 'max', 'min'],
    'SK_DPD': ['mean', 'max'],
    'NAME_CONTRACT_STATUS_Active': ['mean'],
    'NAME_CONTRACT_STATUS_Completed': ['mean']
})
credit_card_agg.columns = [f'credit_card_{col[0].upper()}_{col[1].upper()}' for col in credit_card_agg.columns.values]
credit_card_agg = credit_card_agg.reset_index()


In [43]:
df_train_final = pd.merge(df_train_installments, credit_card_agg, on='SK_ID_CURR', how='left')
for col in credit_card_agg.columns:
    if col != 'SK_ID_CURR':
        df_train_final[col].fillna(0, inplace=True)
df_train_final.head()

/tmp/ipython-input-318875902.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_train_final[col].fillna(0, inplace=True)


,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY_x,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,credit_card_AMT_DRAWINGS_ATM_CURRENT_MEAN,credit_card_AMT_DRAWINGS_ATM_CURRENT_MAX,credit_card_AMT_DRAWINGS_ATM_CURRENT_MIN,credit_card_AMT_DRAWINGS_CURRENT_MEAN,credit_card_AMT_DRAWINGS_CURRENT_MAX,credit_card_AMT_DRAWINGS_CURRENT_MIN,credit_card_SK_DPD_MEAN,credit_card_SK_DPD_MAX,credit_card_NAME_CONTRACT_STATUS_ACTIVE_MEAN,credit_card_NAME_CONTRACT_STATUS_COMPLETED_MEAN
0,100002,1,0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461,-637.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,100003,0,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765,-1188.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,0,67500.0,135000.0,6750.0,135000.0,0.010032,-19046,-225.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,0,135000.0,312682.5,29686.5,297000.0,0.008019,-19005,-3039.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,100007,0,0,121500.0,513000.0,21865.5,513000.0,0.028663,-19932,-3038.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [44]:
# Save the processed DataFrame to a CSV file
output_path = os.path.join(caminho_projeto, 'df_train_final.csv')
df_train_final.to_csv(output_path, index=False)

print(f"Processed DataFrame saved to: {output_path}")

Processed DataFrame saved to: /content/home-credit-default-risk/df_train_final.csv
